## Gait Video Study 
### Validating the estimates 3D poses via CoP computed on the treadmill 
### This code makes (and saves as csv) the dataframe for each video containing their relevant frames, corresponding feet coordinates for these frames and their relative treadmill extracted CoP values 

We will do qualitative and quantitative validation for CoP. 
* First, we need to align the treadmill GaitCycles.csv file to the video time. This will help us align the corresponding video frames to the gait events i.e. HSR/HSL/TOR and TOL to be specific. 
* Once we know for each video, which frame numbers correspond to heel strikes and toe offs, we compute the sequence of frames that are in single support left phase, frames that are in single support right phase and similarly in double support phase. Thus, each frame of a video is labelled to be in SSR, SSL or DS phase. 
* Now, for each frame in DS phase, use the computed real world x, y coordinates for big toe, small toe and heel to make 2 triangular regions for both left and right feet, since in this phase, both feet are on ground and thus impact the center of pressure. Now, plot the corresponding actual COPX, COPY coordinate (as a red dot) for this particular frame. If this red dot lies in the shaded region of computed CoP drawn, we are good to claim that actual CoP lies in the approximate computed CoP region. Similarly, for each frame in the SSL phase, since left foot rests on ground for this phase, the CoP must be determined using the left foot, and hence use the computed x, y coordinates of the left big toe, small toe and heel to draw a shaded triangular region spanned by CoP for this frame, and draw the red dot for the actual x, y of CoP for this frame and if it lies within the shaded region, we are good to claim that actual and computed CoP region match. Now, for each frame in SSR phase, the shaded CoP region must be determined using the right feet's big/small toe and heel's x, y coordinates and if the actual CoP's x, y is bounded in this shaded computed CoP's region, we are good. 

* For qualitative validation, we plot these above mentioned regions for computed CoP and actual CoPs as markers in/out of that region for each video. If we do this for a complete stride, it should follow a butterfly pattern. And hence the inverted triangles and hexagons should occur in a butterfly pattern.
* For quantitative validation, we will call it success (1) if the actual CoP is bounded by the computed CoP shaded region and failure (0) otherwise for every frame of every video for every trial and cohort. 
* Further, for more precise quantitative validation, we can find the lateral, anterior-posterior and euclidean distance of the actual (COP_x, COP_y) with the centroid of our region drawn. This gives us a numerical value quantifying the error in the true and predicted CoP. This step can especially be done for only wrongly predicted values, to further check what is the measure of wrongly predicted values. Further, we may check that we may have error most in the lateral direction or most in the AP direction or eucliean only. 
* Based on the statistics of these success and failure counts, we can quantify the performance of our marker estimation framework using CoP validation. 
* Further, we can try to correlate/have a look at the distribution pattern to relate the correctness of CoP (either quantified using binary scores or using the numerical scores) with the confidence scores predicted by the OpenPose algorithm. Now, since we are only using toes and heel coordinates to draw the CoM trajectory/region, we should only use the confidence scores for heel and toes for this correlation. To be precise, we can average the confidence scores of left feet's heel and 2 toes to get the aggregated confidence score for frames in left single support, similarly, we can average the confidence scores of right feet's heel and 2 toes to get the aggregated confidence score for frames in right single support, and average the confidence scores of both the left and right feet's heel and toes to get the aggregated confidence scores for frames in double support. Now this correlation/relation between the confidence score for each frame and it's coorsponding correctness of CoP metric can be either done on a frame by frame basis. Or rather we can aggreagte all frames over a stride and do the relationship analysis on a stride by stride basis based on some aggregatd confidence scores of stride with some aggregated correctness of CoP score over each stride. 

In [319]:
from importlib import reload
import imports 
reload(imports)
from imports import *

In [320]:
# labels_file = pd.read_csv('C:\\Users\\Rachneet Kaur\\Box\\Gait Video Project\\GaitVideoData\\video\\labels.csv', index_col = 0)
# pd.DataFrame(labels_file.video.unique(), columns = ['video']).to_csv(cop_path+'treamill_video_cop_sync.csv')

In [321]:
#Folder for CoP validation
cop_path = 'C:\\Users\\Rachneet Kaur\\Box\\Gait Video Project\\CoPvalidation\\'
#Subfolder containing the sync files between frame numbers and treadmill identified events for each video
treadmill_video_sync_files = cop_path + 'CoP_treadmill_video_sync\\'
#Path to store the new dataframes to be created for CoP validation 
path_viz_dataframes = cop_path + 'raw_dataframes_for_viz\\'
#Path to log file corresponding to the sync files between frame numbers and treadmill identified events for each video
sync_log_file = cop_path + 'treamill_video_cop_sync.csv'

#Path for reading the frame coordinates and OpenPose confidence scores from (toes and heel in particular)
frame_path = cop_path + '..\\GaitVideoData\\video\\multi_view_merged_data\\' 
#Path for the RAWDATA.csv containing the COP values extracted by the treadmill wrt the time of the walking trial 
cop_treadmill_path = cop_path + '..\\GaitCSVData\\csv\\'

#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD', '\\ExtraHOA']
trials = ['\\beam_walking', '\\walking']

# for every GaitCycle file, a sequence of walk will always start with a heel strike on the right foot.
# Thus the order of the Gait event points would be HSR, TOL, MidSSR, HSL, TOR and MidSSL.
gait_type = np.array(['HSR', 'TOL', 'MidSSR', 'HSL', 'TOR', 'MidSSL'])

trial_dict = {'BW': 'beam_walking', 'W': 'walking'}

In [322]:
# #Need to run only once to create the new sync file 
# #Reading the log of the treadmill and video syncs 
# sync_log = pd.read_csv(sync_log_file, index_col = 0)
# #Setting the new scenario column for marking the video as belonging to one of the W/WT/VBW/VBWT trials 
# sync_log.set_index('video', inplace=True)
# sync_log['scenario'] = labels_file.groupby('video').first()['scenario']
# sync_log.reset_index(inplace=True)
# print ('Total video files: ', sync_log.shape[0])
# #Saving the new sync file with scenario marked 
# sync_log.to_csv(sync_log_file)

### Utility functions 

In [323]:
# Valid strides in the gait_cycles.csv file 
def get_cycle(dataframe):
    stride_start = min(dataframe.loc[dataframe.EventType == 'HSR'].index)
    stride_end = max(dataframe.loc[dataframe.EventType == 'MidSSL'].index)   
    return dataframe.loc[stride_start:stride_end]

In [324]:
# Restore the indexing for the cropped dataframe 
def change_index(dataframe):
    dataframe.index = range(len(dataframe))
    return dataframe

In [325]:
# get all the valid index in order: HSR-TOL-MidSSR-HSL-TOR-MidSSL
def set_complete(data_frame):
    # input is the Dataframe includes ONLY valid points 
    # get all the index of HSR since it starts with heal strike left
    # if the length of last gait cycle contain HSR does not equals to 6, then ignore it
    
    HSR = data_frame.loc[data_frame.EventType == 'HSR'].index
    last_idx = HSR[-1]
    last_all_idx = data_frame.index[-1]
    # if the last gait cycles contains HSR is not a valid gait cycle, then we should consider the last second HSR instead.
    if((last_all_idx-last_idx) < 5):
        HSR = HSR[0:-1] 
    else:
        HSR = HSR
    
    # get all the valid index in order: HSR-TOL-MidSSR-HSL-TOR-MidSSL
    valid = []
    for idx_HSR in HSR:
        if (((idx_HSR + 1) in data_frame.index) & ((idx_HSR + 2) in data_frame.index) &
            ((idx_HSR + 3) in data_frame.index) & ((idx_HSR + 4) in data_frame.index) & 
            ((idx_HSR + 5) in data_frame.index)):
            # the valid index exist in the dataframe.
            if((data_frame.loc[idx_HSR + 1].EventType == 'TOL') & (data_frame.loc[idx_HSR + 2].EventType == 'MidSSR') & 
               (data_frame.loc[idx_HSR + 3].EventType == 'HSL') & (data_frame.loc[idx_HSR + 4].EventType == 'TOR') & 
               (data_frame.loc[idx_HSR + 5].EventType == 'MidSSL')):
                valid.extend(range(idx_HSR, idx_HSR+6))
    #returns the list of valid indices which form complete strides 
    return valid

In [326]:
#Preprocessing the files to delete missing and invalid data 
def cleaning(gaitcycles_dataframe):         
    #Reducing to complete strides data 
    #Making sure we start at the HSR and end at the MidSSL
    gaitcycles_dataframe = get_cycle(gaitcycles_dataframe)
    #Retaining only complete six even strides 
    indices_complete = set_complete(gaitcycles_dataframe)
    gaitcycles_dataframe = gaitcycles_dataframe.loc[indices_complete]

    #Resetting the index 
    gaitcycles_dataframe = change_index(gaitcycles_dataframe)
    #Returning indices to identify consequetive and non-consequetive strides 
    return indices_complete, gaitcycles_dataframe

#### main()

In [327]:
sync_log = pd.read_csv(sync_log_file, index_col = 0)
print ('Total video files: ', sync_log.shape[0])

#Reducing only to video files for which treadmill sync exists 
#(sync time was present in the logs and treadmill software identified valid events)
sync_log = sync_log[sync_log['Sync']=='Exists']
print ('Total video files for which treadmill sync exists (sync time was present in the logs and treadmill software identified valid events): '\
       , sync_log.shape[0])

'''
Columns for the vizualization dataframe for each video 
Support type is left single support, right single support, double support or NaN for the left single support when 
the strides are not consequetive 
'HSR' - 'TOL': Initial double support 
'TOL' - MidSSR' - 'HSL': Right single support
'HSL' - TOR': Terminal double support
'TOR' - MidSSL' - Next 'HSR': Left single support (only when the strides are consequetive, else NaN)
Extracted (x, y) for real world 3D coordinates and OpenPose confidence scores for the feet
Treadmill's COP_x, COP_y for validation 
'''
#We are only extracting feet coordinates to draw center of mass trajectory from the body coordinate files
coords_of_interest = ['left toe 1', 'left toe 2', 'left heel', 'right toe 1', 'right toe 2', 'right heel']

coordinate_cols = ['left toe 1-x', 'left toe 1-y', 'left toe 1-conf', 'left toe 2-x', \
                      'left toe 2-y', 'left toe 2-conf', 'left heel-x', 'left heel-y', 'left heel-conf', 'right toe 1-x', 'right toe 1-y',\
                       'right toe 1-conf', 'right toe 2-x', 'right toe 2-y', 'right toe 2-conf', 'right heel-x', 'right heel-y', \
                       'right heel-conf']
viz_df_column_names = ['Time', 'EventType', 'frame_number', 'support_type'] + coordinate_cols + ['treadmill_COP_x',  'treadmill_COP_y']

Total video files:  107
Total video files for which treadmill sync exists (sync time was present in the logs and treadmill software identified valid events):  102


In [336]:
#For each video with treadmill and video sync available 
for idx in range(len(sync_log)):
    start_time = time.time()
    video = sync_log.iloc[idx]['video']
    cohort = sync_log.iloc[idx]['cohort']
    trial = trial_dict[sync_log.iloc[idx]['trial']]
    
    #For each video, we will create a dataframe and a csv file corresponding to frame coordinates and treadmill CoP
    viz_df = pd.DataFrame(columns = viz_df_column_names)
    #Read the file for the current video syncing event types to frame numbers 
    video_csv = pd.read_excel(treadmill_video_sync_files + video + '.xlsx')
    #Retaining only time for treadmill's CoP matching, event type for SS/DS group assignment 
    #and frame number for extracting body coordinates
    video_csv = video_csv[['Time', 'EventType', 'frame_number']]
    #Dropping the entries/events that could not be synced to their corresponding video frames 
    video_csv.dropna(inplace = True)
    #Retaining the indices and corresponding rows of dataframe for 'complete' 6 event strides only
    #Indices will help identify consequetive and non-consequetive strides 
    indices_retain, video_csv_retain = cleaning(video_csv)
    #Converting frame number to ints 
    video_csv_retain['frame_number'] = video_csv_retain['frame_number'].astype(int)
#     display(video_csv_retain.head(), video_csv.shape, video_csv_retain.shape)
    
    #Filling up the event type, frame number and time columns
    viz_df[['Time', 'EventType', 'frame_number']] = copy.deepcopy(video_csv_retain)
    viz_df.set_index('frame_number', inplace=True)
    #Listing all the frame numbers from the first HSR to the last MidSSL for viz_df 
    viz_df = viz_df.reindex(np.arange(min(viz_df.index), max(viz_df.index)+1))
    #Filling up the time using interpolation (this will indeed follow that each frame is 1/30 seconds apart since our FPS=30)
    viz_df.interpolate(method = 'index', inplace= True)
    viz_df.reset_index(inplace = True)
    
    #Filling up the toe and heel coordinates in cm and confidence scores in the viz dataframe for the current video
    viz_df.set_index('frame_number', inplace = True)
    #To fill up the feet coordinates for each video, setting the path for body coordinate files 
    coordinate_path = frame_path+ cohort + '\\' + trial + '\\' + video + '\\hip_height_normalized\\'
    #Iterating through each frame number to read it's corresponding file for body coordinates and filling them up in the dataframe
    for frame_number in viz_df.index:
        try: #Since some frames from the video data may be missing 
            frame = pd.read_csv(coordinate_path+str(frame_number)+'.csv', index_col = 0)
            #display(frame)
            #For each frame, we are interested in only feet coordinates for drawing the CoM area
            #Further, we use only x, y coordinates and confidence scores for this validation analysis 
            frame_coords_of_interest = frame.loc[coords_of_interest][['x', 'y', 'confidence']] 
            #Filling up the feet coordinates for a particular frame 
            viz_df.loc[frame_number, coordinate_cols] = frame_coords_of_interest.values.flatten()
        except: 
            #If the particular frame was missing in video data, let the values be NaN for that missing video data frame  
            pass 
    #Note that the video extracted coordinates are in 'cm', but the treadmill's CoP are in 'm'
    display(viz_df.head())
    
    #Filling up the treadmill CoP values 
    cop_path = cop_treadmill_path + cohort + '\\' + trial + '\\' + video  + '_RAWDATA.csv'
    cop_file = pd.read_csv(cop_path, header = 1)
    cop_file = cop_file[['Time', 'COPX', 'COPY']]
    display(cop_file.head())
    
    cop_closest_times = [cop_file['Time'][cop_file['Time']>viz_df['Time'].iloc[i]].iloc[0] for i in range(len(viz_df))]
    cop_file.set_index('Time', inplace = True)
    treadmill_COP_x = cop_file.loc[cop_closest_times]['COPX'].values
    treadmill_COP_y = cop_file.loc[cop_closest_times]['COPY'].values
    viz_df['treadmill_COP_x'] = treadmill_COP_x
    viz_df['treadmill_COP_y'] = treadmill_COP_y
    print ('Time: ', time.time()-start_time)
    
    #Filling up the support times 
    
    
    #Handling non-consequetive strides 
    
    break

,Time,EventType,support_type,left toe 1-x,left toe 1-y,left toe 1-conf,left toe 2-x,left toe 2-y,left toe 2-conf,left heel-x,...,right toe 1-y,right toe 1-conf,right toe 2-x,right toe 2-y,right toe 2-conf,right heel-x,right heel-y,right heel-conf,treadmill_COP_x,treadmill_COP_y
frame_number,,,,,,,,,,,,,,,,,,,,,
223,10.4273,HSR,NaN,25.4035,209.01,0.64548,19.5063,208.057,0.669214,21.9412,...,182.185,0.44214,39.6268,179.21,0.447911,28.4004,184.739,0.327902,NaN,NaN
224,10.4608,NaN,NaN,27.5295,207.882,0.634387,20.9282,206.974,0.644836,23.4937,...,181.695,0.405925,40.1646,178.977,0.409807,28.345,186.415,0.317567,NaN,NaN
225,10.4943,NaN,NaN,24.7725,219.151,0.554139,20.3845,217.77,0.569559,27.1475,...,168.917,0.243224,39.823,164.639,0.287539,17.1586,163.038,0.188086,NaN,NaN
226,10.5278,NaN,NaN,24.7779,218.47,0.544383,20.566,217.078,0.558615,27.1676,...,167.877,0.232289,39.8224,164.627,0.272924,17.1763,164.072,0.174737,NaN,NaN
227,10.5613,NaN,NaN,23.6567,220.138,0.563772,20.2807,218.014,0.599672,29.0239,...,166.824,0.403128,39.7904,163.53,0.420511,19.9344,159.717,0.200506,NaN,NaN


,Time,TreadMill_FZ,TreadMill_MX,TreadMill_MY,COPX,COPY,Speed
0,0.000,659.259452,-382.954590,300.029404,0.455101,0.584058,0.028294
1,0.002,657.486195,-381.415728,299.175673,0.455119,0.583968,0.028294
2,0.004,657.465732,-381.441190,299.157045,0.455134,0.583867,0.028294
3,0.006,658.121414,-382.449317,299.378149,0.455147,0.583757,0.028294
4,0.008,656.906157,-379.776986,299.141188,0.455159,0.583639,0.028294


Time:  68.71729779243469


In [360]:
HSR_frames = viz_df[viz_df['EventType'] == 'HSR'].index
TOL_frames = viz_df[viz_df['EventType'] == 'TOL'].index
HSL_frames = viz_df[viz_df['EventType'] == 'HSL'].index
TOR_frames = viz_df[viz_df['EventType'] == 'TOR'].index

In [363]:
[viz_df.loc[HSR_frames[i]:TOL_frames[i]] for i in range(len(HSR_frames))]

[                 Time EventType support_type left toe 1-x left toe 1-y  \
 frame_number                                                             
 223           10.4273       HSR          NaN      25.4035       209.01   
 224           10.4608       NaN          NaN      27.5295      207.882   
 225           10.4943       NaN          NaN      24.7725      219.151   
 226           10.5278       NaN          NaN      24.7779       218.47   
 227           10.5613       NaN          NaN      23.6567      220.138   
 228           10.5948       NaN          NaN      26.6474      215.659   
 229           10.6283       NaN          NaN      26.6478      218.318   
 230           10.6618       NaN          NaN      29.1414      212.153   
 231           10.6953       TOL          NaN      28.3705      211.058   
 
              left toe 1-conf left toe 2-x left toe 2-y left toe 2-conf  \
 frame_number                                                             
 223                  0

In [354]:
viz_df

,Time,EventType,support_type,left toe 1-x,left toe 1-y,left toe 1-conf,left toe 2-x,left toe 2-y,left toe 2-conf,left heel-x,...,right toe 1-y,right toe 1-conf,right toe 2-x,right toe 2-y,right toe 2-conf,right heel-x,right heel-y,right heel-conf,treadmill_COP_x,treadmill_COP_y
frame_number,,,,,,,,,,,,,,,,,,,,,
223,10.427300,HSR,NaN,25.4035,209.01,0.64548,19.5063,208.057,0.669214,21.9412,...,182.185,0.44214,39.6268,179.21,0.447911,28.4004,184.739,0.327902,0.441910,2.115634
224,10.460800,NaN,NaN,27.5295,207.882,0.634387,20.9282,206.974,0.644836,23.4937,...,181.695,0.405925,40.1646,178.977,0.409807,28.345,186.415,0.317567,0.439645,2.124283
225,10.494300,NaN,NaN,24.7725,219.151,0.554139,20.3845,217.77,0.569559,27.1475,...,168.917,0.243224,39.823,164.639,0.287539,17.1586,163.038,0.188086,0.437276,2.135016
226,10.527800,NaN,NaN,24.7779,218.47,0.544383,20.566,217.078,0.558615,27.1676,...,167.877,0.232289,39.8224,164.627,0.272924,17.1763,164.072,0.174737,0.435320,2.150870
227,10.561300,NaN,NaN,23.6567,220.138,0.563772,20.2807,218.014,0.599672,29.0239,...,166.824,0.403128,39.7904,163.53,0.420511,19.9344,159.717,0.200506,0.433838,2.178443
228,10.594800,NaN,NaN,26.6474,215.659,0.520273,21.9339,214.184,0.558653,25.6514,...,164.668,0.309003,39.701,161.406,0.319964,21.343,157.552,0.159402,0.431528,2.210555
229,10.628300,NaN,NaN,26.6478,218.318,0.616182,22.252,216.518,0.619896,26.3382,...,162.537,0.257484,38.9241,159.296,0.270062,20.6357,158.613,0.147233,0.428365,2.244992
230,10.661800,NaN,NaN,29.1414,212.153,0.668512,23.2664,211.172,0.696989,25.6116,...,161.432,0.273908,38.8591,158.211,0.293398,21.2526,155.42,0.173452,0.424508,2.274943
231,10.695300,TOL,NaN,28.3705,211.058,0.658275,22.4715,209.052,0.695444,24.7697,...,190.236,0.405467,27.3204,188.573,0.394317,18.5644,188.165,0.3406,0.420875,2.291203


In [260]:
viz_df['frame_number']

0        223
1        224
2        225
3        226
4        227
5        228
6        229
7        230
8        231
9        232
10       233
11       234
12       235
13       236
14       237
15       238
16       239
17       240
18       241
19       242
20       243
21       244
22       245
23       246
24       247
25       248
26       249
27       250
28       251
29       252
        ... 
1467    1690
1468    1691
1469    1692
1470    1693
1471    1694
1472    1695
1473    1696
1474    1697
1475    1698
1476    1699
1477    1700
1478    1701
1479    1702
1480    1703
1481    1704
1482    1705
1483    1706
1484    1707
1485    1708
1486    1709
1487    1710
1488    1711
1489    1712
1490    1713
1491    1714
1492    1715
1493    1716
1494    1717
1495    1718
1496    1719
Name: frame_number, Length: 1497, dtype: int64

In [259]:
viz_df

,frame_number,Time,EventType,support_type,left toe 1-x,left toe 1-y,left toe 1-conf,left toe 2-x,left toe 2-y,left toe 2-conf,...,right toe 1-y,right toe 1-conf,right toe 2-x,right toe 2-y,right toe 2-conf,right heel-x,right heel-y,right heel-conf,treadmill_COP_x,treadmill_COP_y
0,223,10.427300,HSR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,224,10.460800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,225,10.494300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,226,10.527800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,227,10.561300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,228,10.594800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,229,10.628300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,230,10.661800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,231,10.695300,TOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,232,10.727244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
np.where(np.array(list(map(operator.sub, indices_retain[1:], indices_retain[:-1])))!=1)

(array([41], dtype=int64),)

In [181]:
video_csv_retain.iloc[41]

Time            19.7813
EventType        MidSSL
frame_number        504
Name: 41, dtype: object

In [133]:
x = (video_csv_retain['Time'][1:].values - video_csv_retain['Time'][:-1].values)
x_sorted = np.sort(x)
print (x_sorted)

[0.161 0.161 0.168 0.174 0.176 0.177 0.177 0.177 0.177 0.178 0.178 0.18
 0.18  0.18  0.18  0.18  0.18  0.18  0.18  0.182 0.182 0.182 0.182 0.182
 0.182 0.182 0.182 0.182 0.182 0.182 0.184 0.184 0.184 0.184 0.184 0.184
 0.184 0.184 0.184 0.186 0.186 0.186 0.186 0.186 0.186 0.186 0.186 0.186
 0.186 0.186 0.186 0.186 0.186 0.186 0.186 0.187 0.187 0.187 0.187 0.188
 0.188 0.188 0.189 0.189 0.189 0.189 0.189 0.189 0.189 0.189 0.19  0.19
 0.19  0.19  0.19  0.191 0.191 0.191 0.191 0.191 0.191 0.192 0.192 0.192
 0.192 0.192 0.192 0.192 0.192 0.192 0.192 0.193 0.193 0.194 0.194 0.194
 0.195 0.195 0.196 0.196 0.196 0.197 0.197 0.198 0.198 0.198 0.198 0.198
 0.198 0.198 0.2   0.2   0.2   0.2   0.2   0.2   0.2   0.201 0.201 0.201
 0.201 0.202 0.202 0.202 0.203 0.203 0.204 0.205 0.205 0.205 0.205 0.206
 0.208 0.208 0.209 0.209 0.21  0.21  0.211 0.211 0.211 0.211 0.214 0.214
 0.214 0.216 0.216 0.217 0.217 0.217 0.217 0.22  0.222 0.222 0.222 0.224
 0.225 0.225 0.225 0.225 0.226 0.226 0.23  0.231 0.23

In [246]:
sync_log

,video,Sync,total_frame_count,cohort,trial,scenario
0,GVS_212_T_T1,Exists,1747.0,HOA,BW,SLWT
2,GVS_213_T_T1,Exists,1750.0,HOA,BW,SLWT
3,GVS_213_T_T2,Exists,1748.0,HOA,BW,SLW
4,GVS_214_T_T1,Exists,1824.0,HOA,BW,SLW
5,GVS_214_T_T2,Exists,1757.0,HOA,BW,SLWT
6,GVS_215_T_T1,Exists,1743.0,HOA,BW,SLWT
7,GVS_215_T_T2,Exists,1749.0,HOA,BW,SLW
8,GVS_216_T_T1,Exists,1722.0,HOA,BW,SLWT
9,GVS_216_T_T2,Exists,1738.0,HOA,BW,SLW
10,GVS_217_T_T1,Exists,1745.0,HOA,BW,SLW


In [180]:
1719-223

1496

In [ ]:
#To read the coordinates for a particular frame, use frame_path+HOA\beam_walking\GVS_212_T_T1\hip_height_normalized
#To read the CoP closest to a particular frame's time, use C:\Users\Rachneet Kaur\Box\Gait Video Project\GaitCSVData\csv\HOA\walking\GVS_102_W_T1_RAWDATA.csv